1. csv 파일 불러올 때 encoding
 - 한글 encoding : euc-kr, cp949
 - 엑셀로 불러온다음 인코딩을 강제로 해준다.
 - 컬럼때문에 안불러와지면 컬럼 삭제하고 나중에 가져오면된다.
 - 예외처리를해서 euc-kr, cp949 둘다 해본다.
 
 ```
 try
     euc-kr
 except:
     cp949
 ```
 
2. 1번 코로나확진     2번 따릉이
 - 1번과 2번의 기준이되는 index가 똑같아야한다.
 
 - 1번 코로나 확진 데이터 2번 따릉이
 - index   1번 비교데이터       2번 비교데이터
 - 202001      10                      111
 - 202002      321                     323
 - 202003      4                       67

In [13]:
#데이터 셋의 형태는 이런 구조여야 장바구니 분석이 가능하다.
dataset = [
    ['커피','녹차','얼음'], #1인 구매 리스트
    ['커피','얼음','우유'],
    ['커피','딸기'],
    ['바나나','녹차','딸기'],
    ['커피','얼음'],
    ['우유','커피'],
    ['우유','딸기'],
    ['커피'],
    ['얼음','커피']
]

import pandas as pd
#장바구니 분석을 하기위해 필요한 패키지들
from mlxtend.preprocessing import TransactionEncoder
from mlxtend.frequent_patterns import apriori

In [14]:
#학습
te = TransactionEncoder()
#데이터셋을 fit으로 맞추고 transform으로 바꿔준다.
te_ary = te.fit(dataset).transform(dataset)
te_ary

array([[ True, False, False,  True, False,  True],
       [False, False, False,  True,  True,  True],
       [False,  True, False, False, False,  True],
       [ True,  True,  True, False, False, False],
       [False, False, False,  True, False,  True],
       [False, False, False, False,  True,  True],
       [False,  True, False, False,  True, False],
       [False, False, False, False, False,  True],
       [False, False, False,  True, False,  True]])

In [15]:
#그냥써도 되지만 데이터프레임으로 넣어주면 보기좋아진다.
df = pd.DataFrame(te_ary, columns=te.columns_)
df

,녹차,딸기,바나나,얼음,우유,커피
0,True,False,False,True,False,True
1,False,False,False,True,True,True
2,False,True,False,False,False,True
3,True,True,True,False,False,False
4,False,False,False,True,False,True
5,False,False,False,False,True,True
6,False,True,False,False,True,False
7,False,False,False,False,False,True
8,False,False,False,True,False,True


In [16]:
#apriori 알고리즘 사용
f_data = apriori(df,use_colnames=True)
#min_support옵션을 주지않으면 기본 지지도는 0.5로 설정된다.
#0.2이상만 보여줘
f_data

,support,itemsets
0,0.777778,(커피)


In [17]:
#이번에는 옵션을 넣어 지지도를 낮춰서 본다.
f_data = apriori(df,min_support=0.1, use_colnames=True)
f_data

,support,itemsets
0,0.222222,(녹차)
1,0.333333,(딸기)
2,0.111111,(바나나)
3,0.444444,(얼음)
4,0.333333,(우유)
5,0.777778,(커피)
6,0.111111,"(녹차, 딸기)"
7,0.111111,"(녹차, 바나나)"
8,0.111111,"(녹차, 얼음)"
9,0.111111,"(커피, 녹차)"


In [23]:
from mlxtend.frequent_patterns import association_rules
#지지도(support) 전체거래항목중 a,b를 포함하는 비율
#신뢰도(confidence) a를 포함하는 거래중에 a,b가 동시에 거래되는 비중
#향상도(Lift) a와 b사이에 상호관계가 없으면 1 //// 1보다 큰값 연관성이 낮다

#threshold = 신뢰도
#
#min_threshold의 기본은 0.8이다.
association_rules(f_data, metric='confidence', min_threshold= 0.6)

# association_rules(f_data, metric='lift', min_threshold= 0.3)


,antecedents,consequents,antecedent support,consequent support,support,confidence,lift,leverage,conviction
0,(바나나),(녹차),0.111111,0.222222,0.111111,1.000000,4.500000,0.086420,inf
1,(바나나),(딸기),0.111111,0.333333,0.111111,1.000000,3.000000,0.074074,inf
2,(얼음),(커피),0.444444,0.777778,0.444444,1.000000,1.285714,0.098765,inf
3,(우유),(커피),0.333333,0.777778,0.222222,0.666667,0.857143,-0.037037,0.666667
4,"(녹차, 바나나)",(딸기),0.111111,0.333333,0.111111,1.000000,3.000000,0.074074,inf
5,"(녹차, 딸기)",(바나나),0.111111,0.111111,0.111111,1.000000,9.000000,0.098765,inf
6,"(바나나, 딸기)",(녹차),0.111111,0.222222,0.111111,1.000000,4.500000,0.086420,inf
7,(바나나),"(녹차, 딸기)",0.111111,0.111111,0.111111,1.000000,9.000000,0.098765,inf
8,"(커피, 녹차)",(얼음),0.111111,0.444444,0.111111,1.000000,2.250000,0.061728,inf
9,"(녹차, 얼음)",(커피),0.111111,0.777778,0.111111,1.000000,1.285714,0.024691,inf


### 내가 찾아본 표 분석방법
### lift(향상도) 수치가 1보다 큰 것들이 있는데, 1보다 클수록 우연히 일어나지 않는다는 뜻
### 1이면 아무관계없다는 뜻
### 위의 표를 해석해보면 바나나 구매하면 녹차, 딸기를 구매하는 경우가 있다. 둘의 신뢰도가 0.111111로 같다
### 바나나-녹차 : 0.111111
### 바나나-딸기 : 0.111111
### 이중 녹차와 딸기 둘다 향상도가 1보다 크기때문에 높은 인기를 가진다.